In [1]:
import os

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

from model import simple_mlp as mlp_float
from model_fixed import simple_mlp as mlp_fixed

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
dataset = input_data.read_data_sets('./data', one_hot = False) #read MNIST Dataset


######### float graph build ##########
#model define
model = mlp_float(
    n_layer = 3,
    hidden_dim = 512
)


#placeholder for batch data
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int32, [None,])

#build network graph
logit = model(x)
    
saver = tf.train.Saver()
###### end base model build ######

#define loss and optimizers
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = logit) 
acc = tf.reduce_mean(tf.to_float(tf.equal(tf.to_int32(tf.argmax(logit, axis=-1)), y)))


g = tf.get_default_graph()

#tf.contrib.quantize.create_eval_graph(input_graph = g)

## add trainable quantize graph
# n_w = 8, n_a = 8


_N_W = 4
_N_A = 8


tf.contrib.quantize.experimental_create_training_graph(
    input_graph = g,
    weight_bits = _N_W,
    activation_bits = _N_A
           )


#tf.contrib.quantize.create_training_graph(input_graph = g)



optimizer = tf.train.GradientDescentOptimizer(0.1)
train_op = optimizer.minimize(loss)

print('vars in fixed-point graphs')
for p in tf.global_variables():
    print(p.name)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
vars in fixed-point graphs
w_0:0
b_0:0
w_1:0
b_1:0
w_2:0
b_2:0
w_3:0
b_3:0
xw_plus_b/weights_quant/min:0
xw_plus_b/weights_quant/max:0
xw_plus_b/act_quant/min:0
xw_plus_b/act_quant/max:0
xw_plus_b/act_quant/xw_plus_b/act_quant/min/biased:0
xw_plus_b/act_quant/xw_plus_b/act_quant/min/local_step:0
xw_plus_b/act_quant/xw_plus_b/act_quant/max/biased:0
xw_plus_b/act_quant/xw_plus_

In [3]:
#pretrained model direct quantization test

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #load pretrained model
    saver.restore(sess, './results/model')
    
    n_epoch = 10
    batch_size = 100
    n_iter = int(n_epoch*55000 / batch_size)
    for _ in range(n_iter):
        batch_x, batch_y = dataset.train.next_batch(batch_size)
        sess.run(train_op, feed_dict = {x:batch_x, y: batch_y})
    
    ### test trained model ###
    test_loss, test_acc = sess.run(
            [loss, acc],
            feed_dict = {x: dataset.test.images, y: dataset.test.labels}
            )
    print('after retraining')
    print('test loss : %.4f, acc : %.4f'%(test_loss, test_acc))
    for p in tf.global_variables():
        if 'min' in p.name or 'max' in p.name:
            print(p.name, ' : ', sess.run(p))
    
    w1f, a1f = sess.run(
        [w1_fixed, a1_fixed],
        feed_dict = {x:dataset.test.images}
    )
            

INFO:tensorflow:Restoring parameters from ./results/model
after retraining
test loss : 0.0779, acc : 0.9808
xw_plus_b/weights_quant/min:0  :  -0.21296218
xw_plus_b/weights_quant/max:0  :  0.21353567
xw_plus_b/act_quant/min:0  :  0.0
xw_plus_b/act_quant/max:0  :  3.361193
xw_plus_b/act_quant/xw_plus_b/act_quant/min/biased:0  :  0.0
xw_plus_b/act_quant/xw_plus_b/act_quant/min/local_step:0  :  5501.0
xw_plus_b/act_quant/xw_plus_b/act_quant/max/biased:0  :  3.347507
xw_plus_b/act_quant/xw_plus_b/act_quant/max/local_step:0  :  5501.0
xw_plus_b_1/weights_quant/min:0  :  -0.18098408
xw_plus_b_1/weights_quant/max:0  :  0.18079154
xw_plus_b_1/act_quant/min:0  :  0.0
xw_plus_b_1/act_quant/max:0  :  3.0865178
xw_plus_b_1/act_quant/xw_plus_b_1/act_quant/min/biased:0  :  0.0
xw_plus_b_1/act_quant/xw_plus_b_1/act_quant/min/local_step:0  :  5501.0
xw_plus_b_1/act_quant/xw_plus_b_1/act_quant/max/biased:0  :  3.0739503
xw_plus_b_1/act_quant/xw_plus_b_1/act_quant/max/local_step:0  :  5501.0
xw_plus_b_2/

In [ ]:
tf.reset_default_graph()

w_1
[[-0.02584112 -0.07752335 -0.05168223 ...  0.05168223 -0.05168223
   0.05168223]
 [-0.10336447  0.05168223 -0.05168223 ...  0.10336447 -0.10336447
   0.        ]
 [ 0.07752335  0.05168223 -0.02584112 ...  0.05168223 -0.07752335
   0.        ]
 [-0.05168223 -0.02584112  0.         ... -0.07752335 -0.05168223
  -0.02584112]
 [ 0.02584112  0.07752335  0.         ... -0.05168223 -0.02584112
   0.02584112]]
delta :  0.025841115714285712
/delta :  [[-1.0000001 -3.0000002 -2.0000002 ...  2.0000002 -2.0000002  2.0000002]
 [-4.0000005  2.0000002 -2.0000002 ...  4.0000005 -4.0000005  0.       ]
 [ 3.0000002  2.0000002 -1.0000001 ...  2.0000002 -3.0000002  0.       ]
 [-2.0000002 -1.0000001  0.        ... -3.0000002 -2.0000002 -1.0000001]
 [ 1.0000001  3.0000002  0.        ... -2.0000002 -1.0000001  1.0000001]]
a_1
[[0.42382997 0.         0.         ... 0.3511734  0.3511734  0.24218856]
 [0.         0.16953199 0.         ... 0.14531313 0.         0.7750034 ]
 [0.2664074  0.         0.        